In [10]:
from src.utils.extract_utils import average_vectors, gather_activations_from_dataset

In [11]:
import torch, transformers, accelerate, einops, json


In [17]:
from src.utils.model_utils import load_gpt_model_and_tokeniser

from src.utils.extract_utils import create_steering_vector

from src.utils.intervention_utils import steering_natural_text

In [13]:
model, tokenizer, MODEL_CONFIG = load_gpt_model_and_tokeniser(model_name="meta-llama/Llama-2-7b-hf")

Detected kernel version 5.4.0, which is below the recommended minimum of 5.5.0; this can cause the process to hang. It is recommended to upgrade the kernel to the minimum version or higher.
Loading checkpoint shards: 100%|██████████| 2/2 [00:13<00:00,  6.68s/it]
/root/activation-additions-large-models/myenv/lib/python3.10/site-packages/transformers/utils/hub.py:374: FutureWarning: The `use_auth_token` argument is deprecated and will be removed in v5 of Transformers. Please use `token` instead.
  warnings.warn(


In [14]:
import src.utils.intervention_utils as iu

# Load Datasets

In [15]:
stories = {}

# Open and read the JSON file
with open('datasets/fantasy.json', 'r') as file:
  # Load the JSON data from the file
  dataset_fantasy = json.load(file)

  stories["fantasy"] = dataset_fantasy

with open('datasets/scifi.json', 'r') as file:
  # Load the JSON data from the file
  dataset_scifi = json.load(file)

  stories["scifi"] = dataset_scifi

In [16]:
from src.utils.dataset_utils import read_all_text_files

training_dataset = read_all_text_files("datasets/opentext_subset")

# Cut texts for first 200 tokens
# Determine the cutoff point using the tokenizer
training_dataset = [tokenizer.decode(tokenizer.encode(text)[:200])[4:] for text in training_dataset][:400]

# Creating a Steering Vector

In [9]:
steering_vector = create_steering_vector(
    model,
    tokenizer,
    MODEL_CONFIG,
    dataset_fantasy,
    training_dataset,
    ["layer_hook_names"],
    False,
    False,
    False
)

Gathering activations: 100%|██████████| 400/400 [01:47<00:00,  3.71it/s]


# Try Steering!

In [25]:
# Clearlu should just make num_beams smaller!!!

outputs = steering_natural_text(
    "Here is a story:", 
    28,
    steering_vector[28] * 100, 
    model,
    MODEL_CONFIG, 
    tokenizer, 
    max_new_tokens=100, 
    temperature=1.0, 
    freq_penalty=2.0,
    top_p=0.3,
    n_completions=2,
    n_beams=5
)

  0%|          | 0/2 [00:00<?, ?it/s]

In [24]:
outputs["steered"][1]

'main probably going public top starting really major either heading ( registered heads American kill chief early actually attack at immediate leading ge first general hit because right minimum GET probable very fran get serious national substant worth nicely basically front relative commercial policy incoming two following next big area work flat got after related ($ register deg on priv zeit hits fresh majority — move drama bo coming “ apo same have working(! Mass back excellent up bottom mediarès behaviour support sue (! exec vice┌ hitting finishedkill semantics Public pretty based健 check goes here'

In [ ]:
tokenizer.decode(outputs["steered"][0].squeeze()[-max:])